# Facts and variables

When connecting to an host, ansible creates a dictionary with a huge set of useful informations.

This step is named **fact gathering**. 

You can inspect facts using the `setup` module.

In [1]:
cd exercise-03

/notebooks/exercise-03


In [ ]:
# Let's run the setup module

!ansible -i inventory -m setup localhost 

## Exercise

 - run the following command
 - what does it do?
 - where's  its output ?
 - use the json module to reindent it

In [ ]:
!ansible >/dev/null -i inventory -m setup localhost --tree host_status

In [ ]:
# Solution
!tree host_status
import json
ret = json.load(open('host_status/localhost'))


## Exercise

Gather the following infos from the output of the previous command.

  - hostname
  - first ip address
  - distribution
  - kernel release



In [ ]:
# use this cell for the exercise

## Exercise

modify [this playbook](/edit/notebooks/exercise-03/get-facts.yml) to print:


  - hostname
  - first ip address
  - distribution
  - kernel release


In [ ]:
!ansible-playbook get-facts.yml

## Facts vs Variables

Essentially, facts are per-host variables gathered by ansible.

In the vault lesson we've seen variables in action loaded it via -e @variables.yml.

Now we'll define them with 

```
- hosts: localhost
  vars:
  - one_variable: 1
  - another_variable: 
      that_is: a_map
```

In [ ]:
!cat vars-and-facts.yml

In [ ]:
# This playbook mixes variables and facts
!ansible-playbook vars-and-facts.yml --tags variables

### filters

inside {{ braces }} you can process expressions using filters.

```

  - debug:
      msg: >
        Floor it  {{ ( one + pi ) | int }}


```

In [ ]:
# This playbook mixes variables and facts
!ansible-playbook vars-and-facts.yml --tags filters

## Exercise

Edit [the last section of vars-and-facts.yml](/edit/notebooks/exercise-03/vars-and-facts.yml) so that it shows:
    
    - the free percentage of every device

Hints:
    
    - use the playbook as a reference
    - iterate thru server facts

In [ ]:
!ansible-playbook vars-and-facts.yml --tags exercise

## Conditions and Assertions

Ansible supports basic condition checking via when:
    
```
- hosts: localhost
  tasks:    
  - debug: msg="This always happens"
    when: true
  - debug: msg="This never does"
    when: false
```


You can use `when` with variables and conditions too:

```
- hosts: localhost  
  tasks:    
  - debug: msg="This is {{ansible_kernel}}!"
    when: ansible_system == 'Linux'

```

Remember on `when` and `with_items`

####  `when` statements should not include jinja2 templating delimiters such as {{ }} or {% %}

####  `with_items` [requires proper templating and braces](http://docs.ansible.com/ansible/porting_guide_2.0.html#other-caveats). Do it for  all expressions everywhere except conditionals (`when`):


In [ ]:
!ansible-playbook conditions.yml --tags when

## assertions

The fail module is used to make assertions. Unless told otherwise, ansible playbooks stops when a task fail.

```
- hosts: localhost  
  tags: assert
  tasks:    
  - fail: msg="This is Linux!"
    when: ansible_system != 'Linux'

  - fail: msg="Stop processing now!"
    when: ansible_architecture == 'x86_64'

```

In [ ]:
!ansible-playbook conditions.yml --tags assert

## Exercise

Edit [conditions.yml](/edit/notebooks/exercise-03/conditions.yml) and add a check on free disk size > 75%.

  - get informations from gather_facts
  - iterate thru partition `with_items`
  - use `when` and fail module
    

In [ ]:
!ansible-playbook conditions.yml --tags exercise

## Predefined variables

There is a set of predefined variables too:

  - group_names
  - groups
  - environment 
  
When using playbooks, we'll get this one too:

  - hostvars


In [ ]:
!ansible all[0] -i ../web -m debug -a "var=groups"

### Exercise

Show the content of all 4 variables:

  - is every variable defined?
  - is every variable not empty?

In [ ]:
# Use this cell for the exercise

## Shell environment & Files

Ansible can iterate thru *local* files using

```
- name: >
    Ansible can use globbing only on local system.
  debug:
    msg: >
      I live on the local system {{item}}
  with_fileglob:
  - "/etc/host*"
    

```

AFAIK There's not a remote version of `with_fileglob`, so **you cannot loop over remote fileglob**. This is because:

 - `with_fileglob` is implemented via executing code 
 - ansible won't implicitly execute code on the remote host
 

```
- name: You can implement a remote fileglob in 2 steps
  shell: >
    ls /etc/host*
  register: fileglob_remote
  
- name: Here we go
  debug: msg="{{item}}"
  with_items: fileglob_remote.stdout_lines

```

In [10]:
!ansible-playbook   -i web fileglob.yml --tags fileglob 


PLAY [web] *********************************************************************

TASK [Gathering Facts] *********************************************************
fatal: [172.17.0.3]: UNREACHABLE! => {"changed": false, "msg": "Authentication failure.", "unreachable": true}
fatal: [172.17.0.6]: UNREACHABLE! => {"changed": false, "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.17.0.6 port 22: No route to host\r\n", "unreachable": true}
ok: [172.17.0.5]
ok: [172.17.0.4]

TASK [You can implement a remote fileglob in 2 steps] **************************
changed: [172.17.0.5]
changed: [172.17.0.4]

TASK [Here we go] **************************************************************
ok: [172.17.0.4] => (item=/etc/host.conf) => {
    "item": "/etc/host.conf", 
    "msg": "/etc/host.conf"
}
ok: [172.17.0.4] => (item=/etc/hostname) => {
    "item": "/etc/hostname", 
    "msg": "/etc/hostname"
}
ok: [172.17.0.5] => (item=/etc/host.conf) => {
    "item": "/etc/host.conf", 
    

You can pass `environment` variables (eg: /proc/PID/environ) to ansible plays via

```
- hosts: all
  environment:
    LD_LIBRARY_PATH: /usr/local/lib
  tasks:
    - name: Set at taskgroup level, but you can do it at task level too.
      shell: >
         echo $LD_LIBRARY_PATH
```

In [13]:
!ansible-playbook  -vvv -i web fileglob.yml --tags environment

Using /notebooks/exercise-03/ansible.cfg as config file

PLAYBOOK: fileglob.yml *********************************************************
2 plays in fileglob.yml

PLAY [web] *********************************************************************

TASK [Gathering Facts] *********************************************************
Using module file /usr/local/lib/python2.7/site-packages/ansible/modules/system/setup.py
<172.17.0.3> ESTABLISH SSH CONNECTION FOR USER: None
Using module file /usr/local/lib/python2.7/site-packages/ansible/modules/system/setup.py
Using module file /usr/local/lib/python2.7/site-packages/ansible/modules/system/setup.py
<172.17.0.6> ESTABLISH SSH CONNECTION FOR USER: None
<172.17.0.4> ESTABLISH SSH CONNECTION FOR USER: None
Using module file /usr/local/lib/python2.7/site-packages/ansible/modules/system/setup.py
<172.17.0.5> ESTABLISH SSH CONNECTION FOR USER: None
<172.17.0.3> SSH: EXEC sshpass -d18 ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -o Con

<172.17.0.4> (0, '\r\n{"invocation": {"module_args": {"filter": "*", "gather_subset": ["all"], "fact_path": "/etc/ansible/facts.d", "gather_timeout": 10}}, "changed": false, "ansible_facts": {"ansible_product_serial": "54CK432", "ansible_form_factor": "Portable", "ansible_product_version": "00h", "ansible_fips": false, "ansible_service_mgr": "sshd", "ansible_user_id": "root", "ansible_user_dir": "/root", "ansible_memtotal_mb": 7906, "ansible_ssh_host_key_ecdsa_public": "AAAAE2VjZHNhLXNoYTItbmlzdHAyNTYAAAAIbmlzdHAyNTYAAABBBN5vyysFbIOhxJX8lwCkeIcgq3owAARwyE50PfK2RgtR2w2F8Wi2Vy6YvKuMv6TYsl67p8QgizR6XLfe3jyeEIg=", "ansible_distribution_version": "8.6", "ansible_domain": "", "ansible_virtualization_type": "docker", "ansible_ssh_host_key_ed25519_public": "AAAAC3NzaC1lZDI1NTE5AAAAICIh5s3kPAFJ+pmkVW65x8ix7PbqthQzJwJVfF4BspAs", "ansible_processor_cores": 2, "ansible_virtualization_role": "guest", "ansible_dns": {"nameservers": ["192.168.1.254"], "search": ["lan"]}, "ansible_processor_vcpus": 4,

Using module file /usr/local/lib/python2.7/site-packages/ansible/modules/system/setup.py
<172.17.0.5> ESTABLISH SSH CONNECTION FOR USER: None
<172.17.0.4> SSH: EXEC sshpass -d28 ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -o ConnectTimeout=10 172.17.0.4 '/bin/sh -c '"'"'echo ~ && sleep 0'"'"''
<172.17.0.5> SSH: EXEC sshpass -d29 ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -o ConnectTimeout=10 172.17.0.5 '/bin/sh -c '"'"'echo ~ && sleep 0'"'"''
Using module file /usr/local/lib/python2.7/site-packages/ansible/modules/system/setup.py
<127.0.0.1> ESTABLISH LOCAL CONNECTION FOR USER: root
<127.0.0.1> EXEC /bin/sh -c 'echo ~ && sleep 0'
<127.0.0.1> EXEC /bin/sh -c '( umask 77 && mkdir -p "` echo /root/.ansible/tmp/ansible-tmp-1499382797.11-45688767396533 `" && echo ansible-tmp-1499382797.11-45688767396533="` echo /root/.ansible/tmp/ansible-tmp-1499382797.11-45688767396533 `" ) && sleep 0'
<127.0.0.1> PUT /tmp/tmp6cEuzL TO /root/.ansible/tmp/ansible-tmp

Using module file /usr/local/lib/python2.7/site-packages/ansible/modules/commands/command.py
<172.17.0.4> ESTABLISH SSH CONNECTION FOR USER: None
Using module file /usr/local/lib/python2.7/site-packages/ansible/modules/commands/command.py
<172.17.0.5> ESTABLISH SSH CONNECTION FOR USER: None
Using module file /usr/local/lib/python2.7/site-packages/ansible/modules/commands/command.py
<127.0.0.1> ESTABLISH LOCAL CONNECTION FOR USER: root
<127.0.0.1> EXEC /bin/sh -c 'echo ~ && sleep 0'
<172.17.0.4> SSH: EXEC sshpass -d7 ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -o ConnectTimeout=10 172.17.0.4 '/bin/sh -c '"'"'echo ~ && sleep 0'"'"''
<172.17.0.5> SSH: EXEC sshpass -d32 ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -o ConnectTimeout=10 172.17.0.5 '/bin/sh -c '"'"'echo ~ && sleep 0'"'"''
<127.0.0.1> EXEC /bin/sh -c '( umask 77 && mkdir -p "` echo /root/.ansible/tmp/ansible-tmp-1499382849.21-27122675600316 `" && echo ansible-tmp-1499382849.21-27122675600

ok: [localhost] => {
    "o.stdout": "Benvenuti a Rimini!"
}
META: ran handlers
META: ran handlers

PLAY RECAP *********************************************************************
172.17.0.3                 : ok=0    changed=0    unreachable=1    failed=0   
172.17.0.4                 : ok=4    changed=1    unreachable=0    failed=0   
172.17.0.5                 : ok=4    changed=1    unreachable=0    failed=0   
172.17.0.6                 : ok=0    changed=0    unreachable=1    failed=0   
localhost                  : ok=3    changed=1    unreachable=0    failed=0   

